# Tutorial

## *checkpoint_schedule* application: adjoint-based gradient 
This user tutorial describes an adjoint-based gradient computation using *checkpointing_schedules* package. Therefore, we initally define the adjoint-based gradient problem and then the forward and adjoint solvers prescribed by *checkpointing_schedules* package.

### Defining the application
Let us consider a one-dimensional (1D) problem where it aims to compute the gradient/sensitivity of an objective functional $I$  with respect to a control parameter. The objective functional, is given by the expression:
$$
I(u) = \int_{\Omega} \frac{1}{2} u(x, \tau)u(x, \tau) \, d x
\tag{1}
$$
which measures the energy of a 1D velocity variable $u = u(x, t)$ governed by the 1D viscous Burgers equation, a nonlinear equation for the advection and diffusion on momentum:
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2} = 0.
\tag{2},
$$
where $x \in [0, L]$ is the space variable and $t \in \mathbb{R}^{+}$ represents the time variable. The boundary condition is $u(0, t) = u(L, t) = 0$, where $L$ is the lenght of the 1D domain. The initial condition is given by $u(0, t) = u_0 = \sin(\pi x)$.

The control parameter is the initial conditions $u_0$, thus, the objetive is to compute the adjoint-based gradient of the cost function $I(u)$ with respect to $u_0$. 

This tutorial set the adjoint equation from the continuous formulation, which means the adjoint PDE is obtained from the continuous forward PDE (Partial Differential Equation). Thus, the adjoint-based gradient is given by the expression:
$$
\frac{\partial I}{\partial u_0} \delta u_0 = \int_{\Omega}  u^{\dagger}(x, 0) \delta u_0 \, dx,
\tag{3}
$$
where $u^{\dagger}(x, 0)$ is the adjoint variable governed by the adjoint system:
$$
-\frac{\partial u^{\dagger}}{\partial t} + u^{\dagger} \frac{\partial u}{\partial x} - u \frac{\partial u^{\dagger}}{\partial x} - \nu \frac{\partial^2 u^{\dagger}}{\partial x^2} = 0,
\tag{4}
$$
satisfying the boundary condition $u^{\dagger} (0, t) = u^{\dagger}(L, t) = 0$. In this case, the initial condition is $u^{\dagger} (x, \tau) = u(x, \tau)$.

Compute the adjoint-based gradient requires storing the forward solution for each time-step, since the adjoint equation depends on the forward solution as seen in adjoint equation (4). Additionally, the gradient expression (3) is a function of $u^{\dagger} (0, t)$, which is the final adjoint time 0. 

#### Discretisation
Both the forward and adjoint systems are discretised using the Finite Element Method (FEM), employing a discretisation methodology detailed in [1]. This methodology uses the Galerkin method with linear trial basis functions to obtain an approximate solution. The backward finite difference method is employed to discretise the equations in time.

#### Coding
The *BurgerGradAdj* class is implemented to set of functionalities for solving Burger's equation (2) and its corresponding adjoint equation (4), as well as computing the objective functional (1). The *BurgerGradAdj* class constructor is responsible for defining the spatial and temporal configurations required for solving the problem. It sets up the necessary parameters and initializes the problem domain. The `copy_fwd_data` function carries forward data copying from either RAM or disk. This data is then used as the initial condition in the forward solver restarting. The functions `store_ram` and `store_disk` are responsible for storing the forward data for restarting purposes. Additionally, the `store_adj_deps` function is responsible for storing the forward data required for the adjoint computation.

In [44]:
import functools
from checkpoint_schedules import Forward, EndForward, Reverse, Copy, Move, EndReverse, StorageType

class CheckpointingManager():
    """Manage the forward and backward solvers.

    Attributes
    ----------
    max_n : int
        Total steps used to execute the solvers.
    save_ram : int
        Number of checkpoint that will be stored in RAM.
    save_disk : int
        Number of checkpoint that will be stored on disk.
    forward_solver : function
        The forward solver.
    backward_solver : function
        The backward solver.
    copy_fwd_data : function
    """
    def __init__(self, max_n, save_ram, save_disk):
        self.max_n = max_n
        self.save_ram = save_ram
        self.save_disk = save_disk
        self.forward_solver = None
        self.backward_solver = None
        self.copy_fwd_data = None
        self.move_fwd_data = None
        self.model_r = 0
        
    def execute(self, cp_schedule):
        """Execute forward and adjoint with checkpointing H-Revolve checkpointing method.
        """
        @functools.singledispatch
        def action(cp_action):
            raise TypeError("Unexpected action")

        @action.register(Forward)
        def action_forward(cp_action):
            n1 = min(cp_action.n1, self.max_n)
            self.forward_solver(cp_action)
            
            if n1 == self.max_n:
                cp_schedule.finalize(n1)

        @action.register(Reverse)
        def action_reverse(cp_action):
            self.backward_solver(cp_action)
            self.model_r += cp_action.n1 - cp_action.n0
            
        @action.register(Copy)
        def action_copy(cp_action):
            self.copy_fwd_data(cp_action)

        @action.register(Move)
        def action_move(cp_action):
            self.move_fwd_data(cp_action)
            
        @action.register(EndForward)
        def action_end_forward(cp_action):
            if cp_schedule._max_n is None:
                cp_schedule._max_n = self.max_n
    
            assert self.model_r == 0
            
        @action.register(EndReverse)
        def action_end_reverse(cp_action):
            assert self.model_r == self.max_n

        self.model_r = 0
        for _, cp_action in enumerate(cp_schedule):
            action(cp_action)
            
            if isinstance(cp_action, EndReverse):  
                break

In [45]:
from scipy.sparse import lil_matrix
from scipy.optimize import newton_krylov
import numpy as np
import pickle
from enum import Enum
import os
from scipy.sparse.linalg import spsolve
         
class CheckpointingMethod(Enum):
    NO_REVOLVER = 0
    REVOLVER = 1


class Burger(CheckpointingManager):
    """This class is capable to solve the time-dependent forward 
    and adjoint burger's equation.

    Attributes
    ----------
    model : dict
        The model parameters.
    checkpoints : str, optional
        Checkpointing method. The default is "trivial", which means that
        the forward restart data and the adjoint dependency is stored every time, 
    snapshots : dict
        Storage of the forward restart data. 
    fwd_working_mem : dict
        Store the forward restart data in StorageType.WORK.
    bwd_working_mem : dict
        Store the adjoint restart data in StorageType.WORK.
    adj_deps : dict
        Store the adjoint dependency data in StorageType.WORK.

    """
    def __init__(self, model, checkpointing):
        self.model = model
        self.snapshots = {StorageType.RAM: {}, StorageType.DISK: {}, StorageType.WORKING_MEMORY: {}}
        self.fwd_working_mem = {}
        self.bwd_working_mem = {}
        self.adj_deps = {}
        self.checkpointing = checkpointing
        super().__init__(model["max_n"], model["chk_ram"], model["chk_disk"])
      
    def forward(self, cp_action):
        """Solve the non-linear forward burger's equation in time.

        Parameters
        ----------
        cp_action : object
            The forward checkpoint action. 
        
        Notes
        -----
        This action contains the following attributes: n0, n1, write_ics, storage.
        Where n0 is the initial time step, n1 is the final time step, write_ics is a boolean
        that indicates if the forward restart data will be stored, and storage is the storage type.
        """
        if cp_action.write_ics:
            # Store the forward restart data
            if cp_action.storage == StorageType.DISK:
                self.store_on_disk(self.fwd_working_mem[cp_action.n0], cp_action.n0)
            elif cp_action.storage == StorageType.RAM:
                self.snapshots[cp_action.storage][cp_action.n0] = self.fwd_working_mem[cp_action.n0]
            
        # Get the model parameters
        dx = self.model["lx"] / self.model["nx"]
        nx = self.model["nx"]
        dt = self.model["dt"]
        nu = self.model["nu"]
        b = nu / (dx * dx)
        # Get the number of time steps
        if self.checkpointing == CheckpointingMethod.NO_REVOLVER:
            steps = self.max_n
        else:
            steps = cp_action.n1 - cp_action.n0

        def assemble_matrix_system(u_new, u):
            A = lil_matrix((nx, nx))
            B = lil_matrix((nx, nx))
            B[0, 0] = -1 / 3
            B[0, 1] = -1 / 6
            B[nx - 1, nx - 1] = -1 / 3
            B[nx - 1, nx - 2] = -1 / 6
            A[0, 0] = 1 / 3 - dt * (1/2*u_new[0] / dx + b)
            A[0, 1] = 1 / 6 + dt * (1 / 2 * u_new[0] / dx - b)
            A[nx - 1, nx - 1] = 1 / 3 - dt * (- u_new[nx - 1] / dx + b)
            A[nx - 1, nx - 2] = 1 / 6 + dt * (1 / 2 * u_new[nx - 2] / dx - b)

            for i in range(1, nx - 1):
                B[i, i] = -2 / 3
                B[i, i + 1] = B[i, i - 1] = -1 / 6
                A[i, i - 1] = 1 / 6 - dt * (1 / 2 * u_new[i - 1] / dx + b)
                A[i, i] = 2 / 3 + dt * (1 / 2 * (u_new[i - 1] - u_new[i]) / dx + 2 * b)
                A[i, i + 1] = 1 / 6 + dt * (1 / 2 * u_new[i] / dx - b)

            return A, B

        def non_linear(A, B, u_new, u):
            u[0] = u[nx - 1] = 0
            F = A * u_new + B * u
            return F
        
        def store_data(u, t, storage):
            if storage == StorageType.WORKING_MEMORY:
                self.adj_deps[t] = u
            elif storage == StorageType.DISK:
                self.store_on_disk(u, t)

        # Get the initial condition
        u = self.fwd_working_mem[cp_action.n0]
        self.fwd_working_mem.clear()

        u_new = u.copy()
        t = cp_action.n0 + 1
        while t < steps:
            if (self.checkpointing == CheckpointingMethod.NO_REVOLVER 
                and cp_action.write_adj_deps is True):
                store_data(u, t, cp_action.storage)

            A, B = assemble_matrix_system(u_new, u)
            u_new = newton_krylov(lambda u_new: non_linear(A, B, u_new, u), u)
            u = u_new.copy()
            t += 1

        if cp_action.write_adj_deps:
            if cp_action.storage == StorageType.ADJ_DEPS or StorageType.WORKING_MEMORY:
                self.adj_deps[t] =  u
            elif cp_action.storage == StorageType.DISK:
                self.store_on_disk(u, cp_action.n0, adj_deps=True)

        self.fwd_working_mem[cp_action.n0 + steps] = u

    def backward(self, cp_action):
        """Execute the adjoint equation in time.

        Parameters
        ---------
        cp_action : object
            The adjoint checkpoint action.
        
        Notes
        -----
        This action contains the following attributes: n0, n1, clear_adj_deps.
        Where n0 is the initial time step, n1 is the final time step, and clear_adj_deps is a boolean
        that indicates if the adjoint dependency data will be cleared.

        """
        dx = self.model["lx"] / self.model["nx"]
        nx = self.model["nx"]
        dt = self.model["dt"]
        nu = self.model["nu"]
        b = nu / (dx * dx)

        def assemble_matrix_system(uf):
            A = lil_matrix((nx, nx))
            B = lil_matrix((nx, nx))
            A[0, 0] =  A[nx - 1, nx - 1] = 1 / 3
            A[0, 1] = A[nx - 1, nx - 2] = 1 / 6
            B[0, 0] = 1 / 3 - dt * (uf[0] / dx - b - 1 / 3 * (uf[1] - uf[0]) / dx)
            B[0, 1] = (1 / 6 + dt * (1 / 2 * uf[0] / dx + b - 1 / 6 * (uf[2] - uf[1]) / dx))
            B[nx - 1, nx - 1] = (1 / 3 + dt * (uf[nx - 1] / dx - b 
                                - 1 / 3 * (uf[nx - 1] - uf[nx - 2]) / dx))
            B[nx - 1, nx - 2] = (1 / 6 + dt * (1 / 2 * u_new[nx - 2] / dx 
                                + b - 1 / 6 * (uf[nx - 1] - uf[nx - 2]) / dx))

            for i in range(1, nx - 1):
                v_m = uf[i] / dx
                v_mm1 = uf[i - 1] / dx
                deri = (uf[i] - uf[i - 1]) / dx
                derip = (uf[i + 1] - uf[i]) / dx
                A[i, i - 1] = A[i, i + 1] = 1 / 6
                A[i, i] = 2 / 3
                B[i, i] = 2 / 3 + dt * (1 / 2 * (v_mm1 - v_m) - 2 * b - 2 / 3 * (deri - derip))
                B[i, i - 1] = 1/6 - dt * (1 / 2 * v_mm1 - b - 1 / 6 * deri)
                B[i, i + 1] = 1/6 + dt*(1/2 * v_m + b - 1 / 6 * derip)
            
            return A, B

        if cp_action.n1 == self.max_n:
            self.adj_initcondition(cp_action.n1)

        u = self.bwd_working_mem[cp_action.n1]
        self.bwd_working_mem.clear()

        u_new = np.zeros(nx)
        steps = int(cp_action.n1 - cp_action.n0)
        t = cp_action.n1
        for _ in range(steps):
            u[0] = u[nx - 1] = 0
            uf = (self.adj_deps[t] if self.checkpointing == CheckpointingMethod.NO_REVOLVER 
                  else self.adj_deps[cp_action.n1])
            A, B = assemble_matrix_system(uf)
            d = B.dot(u)
            u_new = spsolve(A, d)
            u = u_new.copy()
            if cp_action.clear_adj_deps:
                if self.checkpointing == CheckpointingMethod.NO_REVOLVER:
                    del self.adj_deps[t]
                else:
                    self.adj_deps.clear() 
            t -= 1
            
        self.bwd_working_mem[cp_action.n0] = u
        
    def copy_data(self, cp_action):
        """Copy the forward data.
        This method is used when the forward restart data is copied from one storage type to another.

        Parameters
        ----------
        cp_action : object
            The copy action that contains the following attributes: n, from_storage, to_storage.
        
        """
        if cp_action.from_storage == StorageType.DISK:
            if cp_action.to_storage == StorageType.FWD_RESTART:
                file_name = self.snapshots[cp_action.from_storage][cp_action.n]
                with open(file_name, "rb") as f:
                    u0 = np.asarray(pickle.load(f), dtype=float)
                self.fwd_working_mem.clear()
                self.fwd_working_mem = {cp_action.n: u0}  
            elif cp_action.to_storage == StorageType.ADJ_DEPS:
                file_name = "adj_deps/fwd_" + str(cp_action.n) + ".pkl"
                with open(file_name, "rb") as f:
                    uf = np.asarray(pickle.load(f), dtype=float)
                self.adj_deps[cp_action.n] = uf         
        else:
            u0 = self.snapshots[cp_action.from_storage][cp_action.n]
            self.fwd_working_mem.clear()
            self.fwd_working_mem = {cp_action.n: u0}  
    
    def move_data(self, cp_action):
        """Move the forward data.
        This method is used when the forward restart data is moved from one storage type to another.

        Parameters
        ----------
        cp_action : object
            The move action that contains the following attributes: n, from_storage, to_storage.
            Where n is the time step, from_storage is the storage type from which the data will be moved,
            and to_storage is the storage type to which the data will be moved.
        """
        if cp_action.from_storage == StorageType.DISK:
            if cp_action.to_storage == StorageType.FWD_RESTART:
                file_name = self.snapshots[cp_action.from_storage][cp_action.n]
                with open(file_name, "rb") as f:
                    u0 = np.asarray(pickle.load(f), dtype=float)
                try:
                    os.remove(file_name)
                    print(f"File '{file_name}' has been successfully deleted.")
                except FileNotFoundError:
                    print(f"File '{file_name}' not found.")

                self.fwd_working_mem.clear() 
                self.fwd_working_mem = {cp_action.n: u0} 
                del self.snapshots[cp_action.from_storage][cp_action.n]
            elif cp_action.to_storage == StorageType.ADJ_DEPS:
                file_name = "adj_deps/fwd_" + str(cp_action.n) + ".pkl"
                with open(file_name, "rb") as f:
                    uf = np.asarray(pickle.load(f), dtype=float)
                try:
                    os.remove(file_name)
                    print(f"File '{file_name}' has been successfully deleted.")
                except FileNotFoundError:
                    print(f"File '{file_name}' not found.")
                self.adj_deps.clear()
                self.adj_deps[cp_action.n] = uf
        else:
            u0 = self.snapshots[cp_action.from_storage][cp_action.n]
            self.fwd_working_mem.clear() 
            self.fwd_working_mem = {cp_action.n: u0} 
            del self.snapshots[cp_action.from_storage][cp_action.n]
        
 
    def compute_grad(self):
        x = np.linspace(0, self.lx, self.nx)
        sens = np.trapz(self.p[0]*1.01*np.sin(np.pi*x), x=x, dx=self.dx)
        print("Sensitivity:", sens)
    
    
    def store_on_disk(self, data, step, adj_deps=False):
        """Store the forward data on disk.

        Parameters
        ----------
        data : array
            The forward data.
        step : int
            The time step.
        adj_deps : bool, optional
            If True, the data is stored in the adjoint dependencies folder.
        """
        if adj_deps:
            file_name = "adj_deps/fwd_"+ str(step) +".pkl"
            with open(file_name, "wb") as f:
                pickle.dump(data, f)
        else:
            file_name = "fwd_data/fwd_"+ str(step) +".pkl"
            with open(file_name, "wb") as f:
                pickle.dump(data, f)
            self.snapshots[StorageType.DISK][step] = file_name

    def fwd_initcondition(self, u0, step):
        """Define the forward initial condition.

        Parameters
        ----------
        u0 : array
            The forward initial condition.
        step : int
            The initial time step.
        """
        self.fwd_working_mem = {step: u0}
    
    def adj_initcondition(self, step):
        """Define the adjoint initial condition.

        Parameters
        ----------
        p0 : array
            The adjoint initial condition.
        step : int
            The initial time step.
        """
        self.bwd_working_mem[step] = self.fwd_working_mem[step]

    def execute_burger(self, cp_schedule):
        self.forward_solver = self.forward
        self.backward_solver = self.backward
        self.copy_fwd_data = self.copy_data
        self.move_fwd_data = self.move_data
        self.execute(cp_schedule)
    

### Using *checkpoint_schedules* package
Analagous to the previous example, we mplement the *CheckpointingManager* class, which provides a management of the forward and adjoint solvers coordinated by the sequence of actions given by the *checkpoint_schedules* package. 

In [46]:
model = {"lx": 1,   # lenght domain
         "nx": 100, # number of nodes
         "dt": 0.01, # time step
         "T": 1, # final time
         "nu": 0.01, # viscosity
         "max_n": 10, # total steps
         "chk_ram": 10, # number of checkpoints in RAM
         "chk_disk": 0, # number of checkpoints on disk
        }

In [47]:
from checkpoint_schedules import SingleStorageSchedule

burger = Burger(model, CheckpointingMethod.NO_REVOLVER) # create the burger's equation object
x = np.linspace(0, model["lx"], model["nx"]) # create the spatial grid
u0 = np.sin(np.pi*x) # initial condition
burger.fwd_initcondition(u0, 0) # set the initial condition
schedule = SingleStorageSchedule() # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 9223372036854775807, True, True, <StorageType.WORKING_MEMORY: 4>)
EndForward()
Reverse(10, 0, True)
EndReverse(False,)


/Users/ddolci/work/checkpoint_schedules/.venv/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:214: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',


In [48]:
from checkpoint_schedules import NoneCheckpointSchedule
burger.fwd_initcondition(u0, 0) # set the initial condition
schedule = NoneCheckpointSchedule() # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 9223372036854775807, False, False, <StorageType.NONE: None>)
EndForward()


In [49]:
model["chk_ram"] = 2 # number of checkpoints in RAM
print(model)

{'lx': 1, 'nx': 100, 'dt': 0.01, 'T': 1, 'nu': 0.01, 'max_n': 10, 'chk_ram': 2, 'chk_disk': 0}


In [50]:
from checkpoint_schedules import Revolve
burger = Burger(model, "revolver") # create the burger's equation object
burger.fwd_initcondition(u0, 0) # set the initial condition
schedule = Revolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 6, True, False, <StorageType.RAM: 0>)
Forward(6, 9, True, False, <StorageType.RAM: 0>)
Forward(9, 10, False, True, <StorageType.ADJ_DEPS: 3>)
EndForward()
Reverse(10, 9, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 8, False, False, <StorageType.FWD_RESTART: 2>)
Forward(8, 9, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(9, 8, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, False, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(8, 7, True)
Move(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(7, 6, True)
Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(0, 3, False, False, <StorageType.FWD_RESTART: 2>)
Forward(3, 5, True, False, <StorageType.RAM: 0>)
Forward(5, 6, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(6, 5, True)
Copy(3, <StorageType.RAM: 0>, <StorageType.FWD_RESTART

In [51]:
from checkpoint_schedules import DiskRevolve
os.makedirs("fwd_data", exist_ok=True) # create a directory to store the forward data
model["chk_ram"] = 1 # number of checkpoints in RAM
burger.fwd_initcondition(u0, 0) # set the initial condition
schedule = DiskRevolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 3, True, False, <StorageType.DISK: 1>)
Forward(3, 6, True, False, <StorageType.DISK: 1>)
Forward(6, 9, True, False, <StorageType.RAM: 0>)
Forward(9, 10, False, True, <StorageType.ADJ_DEPS: 3>)
EndForward()
Reverse(10, 9, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 8, False, False, <StorageType.FWD_RESTART: 2>)
Forward(8, 9, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(9, 8, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, False, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(8, 7, True)
Move(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(7, 6, True)
Copy(3, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(3, 5, True, False, <StorageType.RAM: 0>)
Forward(5, 6, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(6, 5, True)
Copy(3, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)


In [52]:
from checkpoint_schedules import PeriodicDiskRevolve
model["chk_ram"] = 1 # number of checkpoints in RAM
burger.fwd_initcondition(u0, 0) # set the initial condition
schedule = PeriodicDiskRevolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers
...

We use periods of size  3
Forward(0, 3, True, False, <StorageType.DISK: 1>)
Forward(3, 6, True, False, <StorageType.DISK: 1>)
Forward(6, 9, True, False, <StorageType.RAM: 0>)
Forward(9, 10, False, True, <StorageType.ADJ_DEPS: 3>)
EndForward()
Reverse(10, 9, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 8, False, False, <StorageType.FWD_RESTART: 2>)
Forward(8, 9, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(9, 8, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, False, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(8, 7, True)
Move(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(7, 6, True)
Copy(3, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(3, 5, True, False, <StorageType.RAM: 0>)
Forward(5, 6, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(6, 5, True)
Copy(3, <StorageType.RAM: 0>, <Sto

Ellipsis

In [53]:
from checkpoint_schedules import HRevolve
model["chk_disk"] = 1 # number of checkpoints in RAM
burger.fwd_initcondition(u0, 0) # set the initial condition
schedule = HRevolve(model["max_n"], model["chk_ram"], model["chk_disk"]) # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 6, True, False, <StorageType.DISK: 1>)
Forward(6, 9, True, False, <StorageType.RAM: 0>)
Forward(9, 10, False, True, <StorageType.ADJ_DEPS: 3>)
EndForward()
Reverse(10, 9, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 8, False, False, <StorageType.FWD_RESTART: 2>)
Forward(8, 9, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(9, 8, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, False, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(8, 7, True)
Move(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(7, 6, True)
Copy(0, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(0, 3, False, False, <StorageType.FWD_RESTART: 2>)
Forward(3, 5, True, False, <StorageType.RAM: 0>)
Forward(5, 6, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(6, 5, True)
Copy(3, <StorageType.RAM: 0>, <StorageType.FWD_RESTA

In [54]:
from checkpoint_schedules import MultistageCheckpointSchedule
burger.fwd_initcondition(u0, 0) # set the initial condition
model["chk_ram"] = 2
model["chk_disk"] = 0
schedule = MultistageCheckpointSchedule(model["max_n"], model["chk_ram"], model["chk_disk"]) # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 6, True, False, <StorageType.RAM: 0>)
Forward(6, 9, True, False, <StorageType.RAM: 0>)
Forward(9, 10, False, True, <StorageType.ADJ_DEPS: 3>)
EndForward()
Reverse(10, 9, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 8, False, False, <StorageType.FWD_RESTART: 2>)
Forward(8, 9, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(9, 8, True)
Copy(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, False, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(8, 7, True)
Move(6, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(7, 6, True)
Copy(0, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(0, 3, False, False, <StorageType.FWD_RESTART: 2>)
Forward(3, 5, True, False, <StorageType.RAM: 0>)
Forward(5, 6, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(6, 5, True)
Copy(3, <StorageType.RAM: 0>, <StorageType.FWD_RESTART

In [55]:
from checkpoint_schedules import TwoLevelCheckpointSchedule
burger.fwd_initcondition(u0, 0) # set the initial condition
model["chk_ram"] = 4
model["chk_disk"] = 0
period = 3
schedule = TwoLevelCheckpointSchedule(period, model["chk_ram"], binomial_storage=StorageType.RAM) # create the checkpointing schedule
burger.execute_burger(schedule) # execute the forward and adjoint solvers

Forward(0, 3, True, False, <StorageType.DISK: 1>)
Forward(3, 6, True, False, <StorageType.DISK: 1>)
Forward(6, 9, True, False, <StorageType.DISK: 1>)
Forward(9, 12, True, False, <StorageType.DISK: 1>)
EndForward()
Copy(9, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(9, 10, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(10, 9, True)
Copy(6, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, False, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, True, False, <StorageType.RAM: 0>)
Forward(8, 9, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(9, 8, True)
Move(7, <StorageType.RAM: 0>, <StorageType.FWD_RESTART: 2>)
Forward(7, 8, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(8, 7, True)
Copy(6, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(6, 7, False, True, <StorageType.ADJ_DEPS: 3>)
Reverse(7, 6, True)
Copy(3, <StorageType.DISK: 1>, <StorageType.FWD_RESTART: 2>)
Forward(3, 4, False, False, <StorageType.FWD_RESTART: 2>)
Forward(4, 5, True,

In [56]:
# from checkpoint_schedules import MixedCheckpointSchedule
# os.makedirs("adj_deps", exist_ok=True) # create a directory to store the forward data
# burger.fwd_initcondition(u0, 0) # set the initial condition
# model["chk_ram"] = 4
# model["chk_disk"] = 0
# period = 3
# schedule = MixedCheckpointSchedule(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
# burger.execute_burger(schedule) # execute the forward and adjoint solvers